# Replicating SGDNEER by optimizing for currency weights in SGDNEER basket

This code runs, but you need to manually choose the time period over which to optimize. 
Run it from "Start of Optimization" to "If satisfied with optimization results"

This optimizes by returns



#### Official SGDNEER
https://www.mas.gov.sg/statistics/exchange-rates

#### Merchandise Imports By Region/ Market
https://tablebuilder.singstat.gov.sg/table/TS/M450081

#### Domestic Exports By Market
https://tablebuilder.singstat.gov.sg/table/TS/M450101



In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.optimize as sco

import warnings

# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

In [2]:
start = pd.to_datetime('2000-01-01', format='%Y-%m-%d')
today_date = datetime.today().strftime('%Y-%m-%d')

start_analysis = pd.to_datetime('2015-01-01', format='%Y-%m-%d')

In [3]:
tickers = ["USDSGD=X",
          "EURUSD=X",
          "GBPUSD=X",
          "USDJPY=X",
          "USDKRW=X",
          "USDCNY=X",
          "USDTWD=X",
          "USDPHP=X",
          "USDTHB=X",
          "USDMYR=X",
          "USDIDR=X",
          "USDINR=X",
          "AUDUSD=X",
          "USDCHF=X"]

# Clean the official SGDNEER data

In [4]:
# Create an ExcelFile object
excel_file = pd.ExcelFile("S$NEER_202410_2.xlsx")

# Get the list of sheet names
sheet_names = excel_file.sheet_names

# Print the sheet names
print(sheet_names)


['1999-00', '2001-02', '2003-04', '2005-06', '2007-08', '2009-10', '2011-12', '2013-14', '2015-16', '2017-18', '2019-20', '2021-22', '2023-24']


In [5]:
SGDNEER=pd.DataFrame(columns=("AverageforWeekEnding","Index"))

for sheet in sheet_names:
    SGDNEER_year = pd.read_excel("S$NEER_202410_2.xlsx",
                            sheet_name=sheet,
                            header=0,
                            skiprows=4)
    
    SGDNEER_year = SGDNEER_year.dropna(axis=0)
    
    SGDNEER_year.columns = SGDNEER_year.columns.str.replace(' ', '')
    
    SGDNEER = pd.concat([SGDNEER, SGDNEER_year],
                        axis=0,
                        join="outer")
    

In [6]:
SGDNEER

,AverageforWeekEnding,Index
2,1999 Jan 8,100
3,15,99.75
4,22,100.09
5,29,100.18
6,Feb 5,99.76
...,...,...
90,13,139.53
91,20,139.62
92,27,139.74
93,Oct 4,139.74


In [7]:
# Create a date column to record the dates in a proper format

SGDNEER["Date"] = np.nan
SGDNEER.iloc[0,2] = pd.to_datetime(SGDNEER.iloc[0,0])

for row in range(1,len(SGDNEER)):
    SGDNEER.iloc[row,2] = SGDNEER.iloc[row-1,2] + timedelta(days=7)

In [8]:
# Drop the original date column and use the created one as the index

SGDNEER.drop(columns="AverageforWeekEnding",inplace=True)
SGDNEER.set_index("Date",inplace=True)
SGDNEER = SGDNEER.astype(float)

In [9]:
SGDNEER["MAS_Index_Change"] = SGDNEER["Index"]/SGDNEER["Index"].shift(1) - 1

In [10]:
SGDNEER

,Index,MAS_Index_Change
Date,,
1999-01-08,100.00,NaN
1999-01-15,99.75,-0.002500
1999-01-22,100.09,0.003409
1999-01-29,100.18,0.000899
1999-02-05,99.76,-0.004192
...,...,...
2024-09-13,139.53,-0.000358
2024-09-20,139.62,0.000645
2024-09-27,139.74,0.000859


# Read and clean the trade data

### Clean exports data

In [11]:
Exports = pd.read_excel("M450101.xlsx",
                        skiprows=9,
                        skipfooter=23)

In [12]:
Exports.columns = Exports.columns.str.replace('(Thousand Dollars)', '')
Exports.columns = Exports.columns.str.strip()

In [13]:
Exports_mapper = {"United Kingdom":"UK",
                  "Hong Kong":"HK",
                  "Republic Of Korea":"Korea",
                  "Mainland China":"China",
                  "United Arab Emirates":"UAE",
                  "United States":"US"}

In [14]:
Exports.rename(mapper=Exports_mapper,
               axis=1,
               inplace=True)

In [15]:
Exports["Data Series"] = pd.to_datetime(Exports["Data Series"])

In [16]:
Exports.set_index("Data Series",
                  drop=True, 
                  inplace=True)

In [17]:
Exports = Exports.replace('na',np.nan)

In [18]:
Exports.fillna(0, 
               inplace=True)

In [19]:
Exports

,EU,Belgium,Denmark,France,"Germany, Federal Republic Of",Greece,Ireland,Italy,Luxembourg,Netherlands,...,Venezuela,Australia,Fiji,New Caledonia,New Zealand,Papua New Guinea,French Polynesia,Guam,Solomon Islands,Other Countries In Oceania
Data Series,,,,,,,,,,,,,,,,,,,,,
1976-01-01,0.0,4639,3408,10871,29327,5303,117,3596,0.0,10311,...,248,53217,236,93,8788,14459,0.0,0.0,0.0,8890
1976-02-01,0.0,3068,2953,10046,27497,5142,456,4118,0.0,10282,...,130,53115,6141,812,5899,10379,0.0,0.0,0.0,4396
1976-03-01,0.0,4794,3272,13504,35517,5788,620,8605,0.0,12614,...,90,49969,612,8448,7914,13024,0.0,0.0,0.0,5301
1976-04-01,0.0,4630,11284,12843,38008,6042,338,4692,0.0,14644,...,165,42708,163,11653,6752,15890,0.0,0.0,0.0,9183
1976-05-01,0.0,3195,4462,14888,34292,5749,325,5996,0.0,12493,...,233,36182,12475,9357,2286,8132,0.0,0.0,0.0,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,1807972.0,247904,30029,139002,165216,70960,26088,71079,2073.0,584394,...,114,1478694,42006,64313,245003,103017,10644.0,29341.0,14090.0,556858
2024-06-01,1997303.0,567988,44904,114703,176843,84580,94864,69671,4594.0,404151,...,26,1081941,65811,24971,171051,69578,1586.0,27453.0,19024.0,602843
2024-07-01,1743712.0,245800,36214,153187,174281,86787,62872,121597,41277.0,373018,...,221,1278822,51507,17706,268288,112971,8638.0,18604.0,13345.0,506315


### Clean imports data

In [20]:
Imports = pd.read_excel("M450081.xlsx",
                        skiprows=10,
                        skipfooter=23)

In [21]:
Imports.columns = Imports.columns.str.replace('Total Merchandise Imports ->', '')
Imports.columns = Imports.columns.str.strip()

In [22]:
Imports_mapper = {"European Union":"EU",
                  "United Kingdom":"UK",
                  "Hong Kong":"HK",
                  "Republic Of Korea":"Korea",
                  "Mainland China":"China",
                  "United Arab Emirates":"UAE",
                  "United States":"US"}

In [23]:
Imports.rename(mapper=Imports_mapper,
               axis=1,
               inplace=True)

In [24]:
Imports["Data Series"] = pd.to_datetime(Imports["Data Series"])

In [25]:
Imports.set_index("Data Series",
                  drop=True, 
                  inplace=True)

In [26]:
Imports = Imports.replace('na',np.nan)

In [27]:
Imports.fillna(0, 
               inplace=True)

In [28]:
Imports.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 585 entries, 1976-01-01 to 2024-09-01
Columns: 119 entries, Total Merchandise Imports to Commonwealth Of Independent States
dtypes: float64(57), int64(61), object(1)
memory usage: 548.4+ KB


In [29]:
Imports

,Total Merchandise Imports,America,Asia,Europe,Oceania,Africa,EU,Belgium,Denmark,France,...,Kiribati,Niue,Solomon Islands,Tuvalu,Wallis & Fatuna Islands,Micronesia,Palau,South Sudan,Other Countries In Oceania,Commonwealth Of Independent States
Data Series,,,,,,,,,,,,,,,,,,,,,
1976-01-01,1830978,0.0,0.0,0.0,0.0,0.0,0.0,4870,5713,20994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,0.0
1976-02-01,1658989,0.0,0.0,0.0,0.0,0.0,0.0,4183,8234,22398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0
1976-03-01,1781332,0.0,0.0,0.0,0.0,0.0,0.0,6942,8134,26167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0
1976-04-01,1942839,0.0,0.0,0.0,0.0,0.0,0.0,7625,6132,20699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,244,0.0
1976-05-01,1462483,0.0,0.0,0.0,0.0,0.0,0.0,5225,4590,17256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,52768845,8392.7,34545.3,8052.5,1171.8,606.5,5125.1,133564,45896,1442530,...,0.0,11.0,41.0,0.0,0.0,0.0,0.0,48537.0,2068,584854.0
2024-06-01,48899967,7614.8,33001.2,6685.0,1167.5,431.4,4664.1,101553,216442,1356794,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,143.0,2861,398982.0
2024-07-01,53013874,8499.3,34856.1,7797.2,1328.6,532.8,5329.8,127348,78432,1646611,...,1.0,0.0,16.0,0.0,0.0,115.0,0.0,0.0,2894,347171.0


In [30]:
Trade_Partners_list = ["US",
                       "EU",
                       "UK",
                       "Japan",
                       "Korea",
                       "China",
                       "Taiwan",
                       "Philippines",
                       "Thailand",
                       "Malaysia",
                       "Indonesia",
                       "India",
                       "Australia",
                       "HK",
                       "UAE",
                       "Saudi Arabia",
                       "Qatar",
                       "Oman",
                       "Bahrain",
                       "Switzerland"]

In [31]:
Chosen_Exports = Exports[Trade_Partners_list]

In [32]:
Chosen_Exports 

,US,EU,UK,Japan,Korea,China,Taiwan,Philippines,Thailand,Malaysia,Indonesia,India,Australia,HK,UAE,Saudi Arabia,Qatar,Oman,Bahrain,Switzerland
Data Series,,,,,,,,,,,,,,,,,,,,
1976-01-01,195905,0.0,24027,117799,1545,2010,3190,8004,30984,48616,0.0,3599,53217,86187,8175,6135,0,0,694,2491
1976-02-01,103926,0.0,27961,93189,1148,1593,3293,7697,17122,56945,0.0,4642,53115,69975,5265,7151,0,0,2810,2625
1976-03-01,121362,0.0,39757,111956,1916,1112,6866,2908,27649,55032,0.0,5367,49969,58375,4677,6459,0,0,3762,3196
1976-04-01,112178,0.0,28420,115961,1184,2032,4908,1717,32602,63066,0.0,3209,42708,110383,8511,13027,0,0,2452,3825
1976-05-01,116440,0.0,32167,123073,712,1423,3915,3216,31003,51751,0.0,4891,36182,53402,9329,6594,0,0,1978,3294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,2668048,1807972.0,167852,731409,905285,2499020,978222,500222,740803,3043019,2503683.0,618880,1478694,1656512,172285,87039,53929,20015,2882,101964
2024-06-01,2209604,1997303.0,132146,672291,805437,2891876,957320,422926,807085,2458770,1935039.0,540915,1081941,1131792,148263,99667,67207,71129,2277,70947
2024-07-01,3950084,1743712.0,145068,724374,766042,2932496,1160242,506147,792704,2830422,2366457.0,833162,1278822,1203387,172561,91489,55761,11435,5680,111795


In [33]:
Chosen_Imports = Imports[Trade_Partners_list]

In [34]:
# Have to multiply EU numbers by 1000 to match the magnitude of the others

Chosen_Imports["EU"] = Chosen_Imports["EU"]*1000

In [35]:
Chosen_Imports

,US,EU,UK,Japan,Korea,China,Taiwan,Philippines,Thailand,Malaysia,Indonesia,India,Australia,HK,UAE,Saudi Arabia,Qatar,Oman,Bahrain,Switzerland
Data Series,,,,,,,,,,,,,,,,,,,,
1976-01-01,230353,0.0,76180,302616,18364,87683,44996,6523,39659,213953,0.0,10659,46192,41128,93,276066,0.0,0.0,12363,18577
1976-02-01,236759,0.0,77056,247534,10219,44139,39439,4754,38103,233947,0.0,19797,49562,38843,33,324290,0.0,0.0,413,17502
1976-03-01,268103,0.0,82456,315782,15692,55235,34839,5570,43986,235421,0.0,17958,38087,49619,20054,196433,0.0,0.0,14974,17702
1976-04-01,315356,0.0,75699,319166,10975,52569,42933,6433,39455,252143,0.0,17668,77253,36757,142,333462,0.0,0.0,21539,15798
1976-05-01,200794,0.0,60505,234967,12290,30194,43262,6611,29965,247634,0.0,12924,33976,38770,32,155572,0.0,0.0,19448,10282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,6415913,5125100.0,1253888,2523823,3117098,6416919,6345267,435755,1233334,6504396,1794871.0,1169363,1010306,678354,1453333,764934,544501.0,87152.0,1363,1049484
2024-06-01,6317819,4664100.0,887886,2095740,3022160,6110970,7476230,431094,1214378,5278622,1605070.0,988736,1066128,201332,1487427,884777,813908.0,61354.0,96481,685841
2024-07-01,6227433,5329800.0,1352195,2621461,3390705,6713519,6654016,439496,1251268,6082492,1658934.0,1168837,1233425,240517,1507017,541134,777533.0,159129.0,143734,697985


### Get trade weights and currency weights

In [36]:
Trade_Weights = Chosen_Exports + Chosen_Imports

In [37]:
Trade_Weights["Total"] = Trade_Weights.sum(axis=1)

In [38]:
Trade_Weights

,US,EU,UK,Japan,Korea,China,Taiwan,Philippines,Thailand,Malaysia,...,India,Australia,HK,UAE,Saudi Arabia,Qatar,Oman,Bahrain,Switzerland,Total
Data Series,,,,,,,,,,,,,,,,,,,,,
1976-01-01,426258,0.0,100207,420415,19909,89693,48186,14527,70643,262569,...,14258,99409,127315,8268,282201,0.0,0.0,13057,21068,2017983.0
1976-02-01,340685,0.0,105017,340723,11367,45732,42732,12451,55225,290892,...,24439,102677,108818,5298,331441,0.0,0.0,3223,20127,1840847.0
1976-03-01,389465,0.0,122213,427738,17608,56347,41705,8478,71635,290453,...,23325,88056,107994,24731,202892,0.0,0.0,18736,20898,1912274.0
1976-04-01,427534,0.0,104119,435127,12159,54601,47841,8150,72057,315209,...,20877,119961,147140,8653,346489,0.0,0.0,23991,19623,2163531.0
1976-05-01,317234,0.0,92672,358040,13002,31617,47177,9827,60968,299385,...,17815,70158,92172,9361,162166,0.0,0.0,21426,13576,1616596.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,9083961,6933072.0,1421740,3255232,4022383,8915939,7323489,935977,1974137,9547415,...,1788243,2489000,2334866,1625618,851973,598430.0,107167.0,4245,1151448,68662889.0
2024-06-01,8527423,6661403.0,1020032,2768031,3827597,9002846,8433550,854020,2021463,7737392,...,1529651,2148069,1333124,1635690,984444,881115.0,132483.0,98758,756788,63893988.0
2024-07-01,10177517,7073512.0,1497263,3345835,4156747,9646015,7814258,945643,2043972,8912914,...,2001999,2512247,1443904,1679578,632623,833294.0,170564.0,149414,809780,69872470.0


In [39]:
Currency_Weights = pd.DataFrame(index = Trade_Weights.index, 
                                columns = ["USD","EUR","GBP","JPY","KRW","CNY","TWD","PHP","THB","MYR","IDR","INR","AUD","CHF"] )

In [40]:
# Weights of all the currencies pegged to USD will just be transferred to USD

Currency_Weights ["USD"] = Trade_Weights[["US",
                                         "HK",
                                         "UAE",
                                         "Saudi Arabia",
                                         "Qatar",
                                         "Oman",
                                         "Bahrain"]].sum(axis=1)/Trade_Weights["Total"]

In [41]:
Currency_Weights ["EUR"] = Trade_Weights["EU"]/Trade_Weights["Total"]
Currency_Weights ["GBP"] = Trade_Weights["UK"]/Trade_Weights["Total"]
Currency_Weights ["JPY"] = Trade_Weights["Japan"]/Trade_Weights["Total"]
Currency_Weights ["KRW"] = Trade_Weights["Korea"]/Trade_Weights["Total"]
Currency_Weights ["CNY"] = Trade_Weights["China"]/Trade_Weights["Total"]
Currency_Weights ["TWD"] = Trade_Weights["Taiwan"]/Trade_Weights["Total"]
Currency_Weights ["PHP"] = Trade_Weights["Philippines"]/Trade_Weights["Total"]
Currency_Weights ["THB"] = Trade_Weights["Thailand"]/Trade_Weights["Total"]
Currency_Weights ["MYR"] = Trade_Weights["Malaysia"]/Trade_Weights["Total"]
Currency_Weights ["IDR"] = Trade_Weights["Indonesia"]/Trade_Weights["Total"]
Currency_Weights ["INR"] = Trade_Weights["India"]/Trade_Weights["Total"]
Currency_Weights ["AUD"] = Trade_Weights["Australia"]/Trade_Weights["Total"]
Currency_Weights ["CHF"] = Trade_Weights["Switzerland"]/Trade_Weights["Total"]

In [42]:
# To confirm that currency weights sum up to 1

# Currency_Weights["Total"] = Currency_Weights.sum(axis=1)
# Currency_Weights

In [43]:
# Rolling 12 month weights to avoid month to month fluctuations

Currency_Weights_12M = Currency_Weights.rolling(12).mean()

Currency_Weights_12M

,USD,EUR,GBP,JPY,KRW,CNY,TWD,PHP,THB,MYR,IDR,INR,AUD,CHF
Data Series,,,,,,,,,,,,,,
1976-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.213116,0.104072,0.018080,0.047402,0.060714,0.146645,0.110387,0.014335,0.030700,0.123244,0.057639,0.023207,0.029392,0.021067
2024-06-01,0.211896,0.103429,0.018186,0.046770,0.060832,0.146082,0.112828,0.013948,0.030957,0.122554,0.057954,0.023204,0.030439,0.020923
2024-07-01,0.211757,0.102585,0.018155,0.046835,0.061003,0.145470,0.113268,0.013831,0.030765,0.123444,0.058227,0.023926,0.031018,0.019715


### Get FX prices from yfinance

In [44]:
FX_prices = yf.download(tickers, 
                     start=start, 
                     end=today_date,
                     period = "1d")

[*********************100%%**********************]  14 of 14 completed


In [45]:
FX_prices = FX_prices["Close"].dropna()

In [46]:
# yfinance has some issues with Asia FX prices
# Sanity check for prices, if prices move more than 50% in a day, just use previous value

for row in range(1,len(FX_prices)):
    for column in range(FX_prices.shape[1]):
        if abs(FX_prices.iloc[row, column]/FX_prices.iloc[row-1, column]-1)>=0.5:
            FX_prices.iloc[row, column] = FX_prices.iloc[row-1, column]
        else:
            continue

In [47]:
FX_prices

Ticker,AUDUSD=X,EURUSD=X,GBPUSD=X,USDCHF=X,USDCNY=X,USDIDR=X,USDINR=X,USDJPY=X,USDKRW=X,USDMYR=X,USDPHP=X,USDSGD=X,USDTHB=X,USDTWD=X
Date,,,,,,,,,,,,,,
2006-05-16,0.767106,1.285000,1.887398,1.20780,7.9930,9188.599609,44.994999,109.760002,929.969971,3.5927,51.860001,1.57400,37.959000,31.802999
2006-05-17,0.759417,1.274194,1.882495,1.21450,7.9910,8978.900391,44.625000,110.910004,925.289978,3.5696,51.860001,1.58000,38.240002,31.572001
2006-05-18,0.764000,1.285099,1.893294,1.20500,8.0004,9124.900391,45.400002,110.900002,933.809998,3.5874,51.860001,1.57820,38.050999,31.851000
2006-05-19,0.757174,1.276699,1.880017,1.21750,8.0100,9189.200195,45.294998,111.660004,933.020020,3.6129,52.549999,1.58320,38.143002,31.950001
2006-05-22,0.753409,1.285298,1.885085,1.20530,8.0160,9242.700195,45.290001,111.459999,939.400024,3.6276,52.580002,1.58450,38.312000,32.139999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-17,0.666489,1.086248,1.299022,0.86530,7.1184,15583.900391,84.039299,149.576004,1364.199951,4.2925,57.782001,1.31335,33.176998,32.134998
2024-10-18,0.669972,1.082954,1.301575,0.86593,7.1230,15493.000000,84.053802,150.100006,1370.050049,4.3090,57.689999,1.31417,33.144001,32.068001
2024-10-21,0.671682,1.086862,1.305262,0.86493,7.1008,15467.000000,84.068703,149.600006,1369.010010,4.2960,57.509998,1.30998,33.090000,32.042000


In [48]:
# Plot prices to check
fig = px.line(
    FX_prices, 
    x=FX_prices.index, 
    y=["USDTWD=X"],
    labels={"value": "Index Value", "variable": "Index Type"},
    title="Checking yfinance prices"
)

# Show the plot
fig.show()

In [49]:
SGD_base_prices = pd.DataFrame(index = FX_prices.index,
                               columns = ("SGDUSD",
                                          "SGDEUR",
                                          "SGDGBP",
                                          "SGDJPY",
                                          "SGDKRW",
                                          "SGDCNY",
                                          "SGDTWD",
                                          "SGDPHP",
                                          "SGDTHB",
                                          "SGDMYR",
                                          "SGDIDR",
                                          "SGDINR",
                                          "SGDAUD",
                                          "SGDCHF")
                               )

In [50]:
SGD_base_prices["SGDUSD"] = 1 / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDEUR"] = (1 / FX_prices ["EURUSD=X"]) / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDGBP"] = (1 / FX_prices ["GBPUSD=X"]) / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDJPY"] = FX_prices ["USDJPY=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDKRW"] = FX_prices ["USDKRW=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDCNY"] = FX_prices ["USDCNY=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDTWD"] = FX_prices ["USDTWD=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDPHP"] = FX_prices ["USDPHP=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDTHB"] = FX_prices ["USDTHB=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDMYR"] = FX_prices ["USDMYR=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDIDR"] = FX_prices ["USDIDR=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDINR"] = FX_prices ["USDINR=X"] / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDAUD"] = (1 / FX_prices ["AUDUSD=X"]) / FX_prices ["USDSGD=X"]
SGD_base_prices["SGDCHF"] = FX_prices ["USDCHF=X"] / FX_prices ["USDSGD=X"]

In [51]:
SGD_base_prices

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2006-05-16,0.635324,0.494415,0.336614,69.733165,590.832255,5.078145,20.205209,32.947904,24.116264,2.282529,5837.737995,28.586403,0.828208,0.767344
2006-05-17,0.632911,0.496715,0.336209,70.196203,585.626552,5.057595,19.982278,32.822784,24.202532,2.259241,5682.848194,28.243670,0.833418,0.768671
2006-05-18,0.633633,0.493062,0.334672,70.269930,591.693073,5.069319,20.181853,32.860221,24.110378,2.273096,5781.840382,28.766951,0.829363,0.763528
2006-05-19,0.631632,0.494739,0.335971,70.528048,589.325437,5.059374,20.180648,33.192269,24.092346,2.282024,5804.194242,28.609777,0.834197,0.769012
2006-05-22,0.631114,0.491026,0.334793,70.343958,592.868445,5.059009,20.284001,33.183972,24.179237,2.289429,5833.196880,28.583151,0.837677,0.760682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-17,0.761412,0.700956,0.586142,113.888916,1038.717775,5.420033,24.467963,43.995891,25.261354,3.268360,11865.763763,63.988504,1.142422,0.658850
2024-10-18,0.760937,0.702649,0.584628,114.216582,1042.521170,5.420151,24.401714,43.898429,25.220482,3.278876,11789.190109,63.959611,1.135774,0.658918
2024-10-21,0.763370,0.702362,0.584841,114.200218,1045.061736,5.420541,24.459915,43.901431,25.259927,3.279439,11807.050175,64.175560,1.136506,0.660262


In [52]:
# Plot prices to check
fig = px.line(
    SGD_base_prices, 
    x=SGD_base_prices.index, 
    y=["SGDUSD"],
    labels={"value": "Index Value", "variable": "Index Type"},
    title="Checking yfinance prices"
)

# Show the plot
fig.show()

In [53]:
SGDNEER_index = SGDNEER[SGDNEER.index>=start_analysis]
SGDNEER_index.rename(columns={"Index":"MAS_Index"},
                     inplace=True)

In [54]:
SGDNEER_index

,MAS_Index,MAS_Index_Change
Date,,
2015-01-02,122.80,-0.001139
2015-01-09,122.62,-0.001466
2015-01-16,122.65,0.000245
2015-01-23,122.79,0.001141
2015-01-30,122.30,-0.003991
...,...,...
2024-09-13,139.53,-0.000358
2024-09-20,139.62,0.000645
2024-09-27,139.74,0.000859


In [55]:
SGD_base_prices

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2006-05-16,0.635324,0.494415,0.336614,69.733165,590.832255,5.078145,20.205209,32.947904,24.116264,2.282529,5837.737995,28.586403,0.828208,0.767344
2006-05-17,0.632911,0.496715,0.336209,70.196203,585.626552,5.057595,19.982278,32.822784,24.202532,2.259241,5682.848194,28.243670,0.833418,0.768671
2006-05-18,0.633633,0.493062,0.334672,70.269930,591.693073,5.069319,20.181853,32.860221,24.110378,2.273096,5781.840382,28.766951,0.829363,0.763528
2006-05-19,0.631632,0.494739,0.335971,70.528048,589.325437,5.059374,20.180648,33.192269,24.092346,2.282024,5804.194242,28.609777,0.834197,0.769012
2006-05-22,0.631114,0.491026,0.334793,70.343958,592.868445,5.059009,20.284001,33.183972,24.179237,2.289429,5833.196880,28.583151,0.837677,0.760682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-17,0.761412,0.700956,0.586142,113.888916,1038.717775,5.420033,24.467963,43.995891,25.261354,3.268360,11865.763763,63.988504,1.142422,0.658850
2024-10-18,0.760937,0.702649,0.584628,114.216582,1042.521170,5.420151,24.401714,43.898429,25.220482,3.278876,11789.190109,63.959611,1.135774,0.658918
2024-10-21,0.763370,0.702362,0.584841,114.200218,1045.061736,5.420541,24.459915,43.901431,25.259927,3.279439,11807.050175,64.175560,1.136506,0.660262


In [56]:
SGD_base_prices.loc[:"2015-01-02", "SGDUSD"]

Date
2006-05-16    0.635324
2006-05-17    0.632911
2006-05-18    0.633633
2006-05-19    0.631632
2006-05-22    0.631114
                ...   
2014-12-29    0.755607
2014-12-30    0.755344
2014-12-31    0.756808
2015-01-01    0.755344
2015-01-02    0.755413
Name: SGDUSD, Length: 2220, dtype: float64

In [57]:
SGD_base_prices.loc[:"2015-01-02", "SGDUSD"].rolling(5).mean()

Date
2006-05-16         NaN
2006-05-17         NaN
2006-05-18         NaN
2006-05-19         NaN
2006-05-22    0.632923
                ...   
2014-12-29    0.756161
2014-12-30    0.755651
2014-12-31    0.756103
2015-01-01    0.755845
2015-01-02    0.755703
Name: SGDUSD, Length: 2220, dtype: float64

### Aggregate from daily prices to weekly average prices

In [58]:
SGD_weekly_prices = pd.DataFrame(index = SGDNEER_index.index,
                                 columns = SGD_base_prices.columns)

for row in range (len(SGD_weekly_prices)):
    for column in range (SGD_weekly_prices.shape[1]):
        
        # Get the week and currency pair
        Week = SGD_weekly_prices.index[row]
        Currency_Pair = SGD_weekly_prices.columns[column]
        
        # Extract the relevant data for rolling mean (up to the closest date)
        data_series = SGD_base_prices.loc[:Week, Currency_Pair]
        
        # Compute the 5-period rolling mean and assign the last value to the cell
        SGD_weekly_prices.iloc[row, column] = data_series.rolling(5).mean().iloc[-1]

In [59]:
# Make sure the prices are in numerical format

SGD_weekly_prices = SGD_weekly_prices.astype(float)
SGD_weekly_prices

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,0.755703,0.622771,0.485677,90.692830,829.157130,4.685658,23.711742,33.731107,24.850528,2.647120,9372.988792,47.864281,0.926430,0.748833
2015-01-09,0.748779,0.630373,0.493429,89.495932,824.993539,4.644524,23.900213,33.599660,24.642365,2.664821,9445.061060,47.341743,0.925851,0.757211
2015-01-16,0.751080,0.637588,0.494486,88.303707,810.831934,4.648096,23.919302,33.510110,24.601708,2.679855,9423.570601,46.647360,0.917856,0.738852
2015-01-23,0.749241,0.649595,0.495942,88.489024,810.731095,4.646879,23.664227,33.220876,24.407953,2.692337,9386.917861,46.198425,0.920300,0.650988
2015-01-30,0.742332,0.658627,0.491677,87.520473,805.219798,4.627982,23.293982,32.655953,24.200326,2.682112,9262.013356,45.646718,0.942211,0.669727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.767042,0.694354,0.585917,109.224093,1027.412265,5.454986,24.594652,43.002832,25.828441,3.328032,11844.108876,64.412008,1.148765,0.650833
2024-09-20,0.771908,0.693940,0.584792,109.482223,1022.225165,5.469065,24.590996,42.996791,25.678372,3.290153,11828.772396,64.661970,1.141836,0.653604
2024-09-27,0.776750,0.696363,0.581221,111.908278,1031.255034,5.464206,24.691751,43.328364,25.411559,3.234582,11763.127188,64.913858,1.133500,0.658140


In [60]:
# Calculate log return of the average weekly prices

SGD_weekly_returns = \
    (
        np.log(SGD_weekly_prices/SGD_weekly_prices.shift(1))
    )
    
SGD_weekly_returns

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09,-0.009205,0.012132,0.015837,-0.013285,-0.005034,-0.008818,0.007917,-0.003905,-0.008412,0.006665,0.007660,-0.010977,-0.000626,0.011126
2015-01-16,0.003069,0.011381,0.002139,-0.013411,-0.017315,0.000769,0.000798,-0.002669,-0.001651,0.005626,-0.002278,-0.014776,-0.008673,-0.024544
2015-01-23,-0.002452,0.018657,0.002941,0.002096,-0.000124,-0.000262,-0.010721,-0.008669,-0.007907,0.004647,-0.003897,-0.009671,0.002659,-0.126606
2015-01-30,-0.009264,0.013808,-0.008637,-0.011006,-0.006821,-0.004075,-0.015769,-0.017151,-0.008543,-0.003805,-0.013396,-0.012014,0.023529,0.028378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.000545,0.002688,0.004362,-0.017795,0.003375,0.001287,0.003364,-0.003230,-0.008377,-0.000007,-0.002462,0.000938,0.010671,0.000749
2024-09-20,0.006323,-0.000596,-0.001923,0.002361,-0.005061,0.002577,-0.000149,-0.000140,-0.005827,-0.011447,-0.001296,0.003873,-0.006050,0.004248
2024-09-27,0.006253,0.003485,-0.006125,0.021917,0.008795,-0.000889,0.004089,0.007682,-0.010445,-0.017034,-0.005565,0.003888,-0.007327,0.006917


In [61]:
Currency_Weights_12M

,USD,EUR,GBP,JPY,KRW,CNY,TWD,PHP,THB,MYR,IDR,INR,AUD,CHF
Data Series,,,,,,,,,,,,,,
1976-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.213116,0.104072,0.018080,0.047402,0.060714,0.146645,0.110387,0.014335,0.030700,0.123244,0.057639,0.023207,0.029392,0.021067
2024-06-01,0.211896,0.103429,0.018186,0.046770,0.060832,0.146082,0.112828,0.013948,0.030957,0.122554,0.057954,0.023204,0.030439,0.020923
2024-07-01,0.211757,0.102585,0.018155,0.046835,0.061003,0.145470,0.113268,0.013831,0.030765,0.123444,0.058227,0.023926,0.031018,0.019715


In [62]:
SGD_weights = pd.DataFrame(index = SGD_weekly_returns.index,
                           columns = SGD_weekly_returns.columns)

for row in range(len(SGD_weights)):
    for row_1 in range(len(Currency_Weights_12M)-1):
        if (SGD_weights.index[row] >= Currency_Weights_12M.index[row_1]) & (SGD_weights.index[row] < Currency_Weights_12M.index[row_1+1]):
            SGD_weights.iloc[row,:] = Currency_Weights_12M.iloc[row_1,:]
            break
        else: 
            SGD_weights.iloc[row,:] = Currency_Weights_12M.iloc[-1,:]

In [63]:
SGD_weights

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-09,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-16,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-23,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-30,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785
2024-09-20,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785
2024-09-27,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785


In [64]:
SGD_weekly_returns

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09,-0.009205,0.012132,0.015837,-0.013285,-0.005034,-0.008818,0.007917,-0.003905,-0.008412,0.006665,0.007660,-0.010977,-0.000626,0.011126
2015-01-16,0.003069,0.011381,0.002139,-0.013411,-0.017315,0.000769,0.000798,-0.002669,-0.001651,0.005626,-0.002278,-0.014776,-0.008673,-0.024544
2015-01-23,-0.002452,0.018657,0.002941,0.002096,-0.000124,-0.000262,-0.010721,-0.008669,-0.007907,0.004647,-0.003897,-0.009671,0.002659,-0.126606
2015-01-30,-0.009264,0.013808,-0.008637,-0.011006,-0.006821,-0.004075,-0.015769,-0.017151,-0.008543,-0.003805,-0.013396,-0.012014,0.023529,0.028378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.000545,0.002688,0.004362,-0.017795,0.003375,0.001287,0.003364,-0.003230,-0.008377,-0.000007,-0.002462,0.000938,0.010671,0.000749
2024-09-20,0.006323,-0.000596,-0.001923,0.002361,-0.005061,0.002577,-0.000149,-0.000140,-0.005827,-0.011447,-0.001296,0.003873,-0.006050,0.004248
2024-09-27,0.006253,0.003485,-0.006125,0.021917,0.008795,-0.000889,0.004089,0.007682,-0.010445,-0.017034,-0.005565,0.003888,-0.007327,0.006917


In [65]:
# Create a column to mimic the MAS SGDNEER index and use the first MAS value as the value of this index

SGDNEER_index["Matched_Index"]=np.nan
SGDNEER_index.iloc[0,2]=SGDNEER_index.iloc[0,0]

In [66]:
# For every week, multiply the weekly returns with the weights to get the change in SGDNEER

for row in range(1,len(SGDNEER_index)):
    SGDNEER_index.iloc[row,-1] = \
        (
            SGDNEER_index.iloc[row-1,-1] 
            * 
            (
                1+(sum(SGD_weekly_returns.iloc[row,:] * SGD_weights.iloc[row,:]))
            )
        )

In [67]:
sum(SGD_weekly_returns.iloc[1,:] * SGD_weights.iloc[1,:])

-0.001023400478531556

In [68]:
SGDNEER_index

,MAS_Index,MAS_Index_Change,Matched_Index
Date,,,
2015-01-02,122.80,-0.001139,122.800000
2015-01-09,122.62,-0.001466,122.674326
2015-01-16,122.65,0.000245,122.633329
2015-01-23,122.79,0.001141,122.382861
2015-01-30,122.30,-0.003991,121.849231
...,...,...,...
2024-09-13,139.53,-0.000358,139.687601
2024-09-20,139.62,0.000645,139.646653
2024-09-27,139.74,0.000859,139.763255


In [69]:
# Plot MAS_Index and Matched_Index using Plotly

fig = px.line(
    SGDNEER_index, 
    x=SGDNEER_index.index, 
    y=["MAS_Index", "Matched_Index"],
    labels={"value": "Index Value", "variable": "Index Type"},
    title="MAS Index vs Matched Index"
)

# Show the plot
fig.show()

In [70]:
SGD_optimized_overall_weights = pd.DataFrame(index = SGD_weights.index, 
                                             columns = SGD_weights.columns)

# Start of optimization

Period 1: 2015-01-01 - 2016-10-31

Period 2: 2016-10-31 - 2022-10-14

Period 3: 2022-10-14 - 2024-10-31

In [71]:
# Define the Optimization Period

optimization_start = pd.to_datetime('2015-01-01', format='%Y-%m-%d')
optimization_end = pd.to_datetime('2024-10-31', format='%Y-%m-%d')

In [72]:
# Select Weekly Returns for the Optimization Period

SGD_optimization_weekly_returns = SGD_weekly_returns.loc[optimization_start:optimization_end,:]

In [73]:
SGD_optimization_weekly_returns

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09,-0.009205,0.012132,0.015837,-0.013285,-0.005034,-0.008818,0.007917,-0.003905,-0.008412,0.006665,0.007660,-0.010977,-0.000626,0.011126
2015-01-16,0.003069,0.011381,0.002139,-0.013411,-0.017315,0.000769,0.000798,-0.002669,-0.001651,0.005626,-0.002278,-0.014776,-0.008673,-0.024544
2015-01-23,-0.002452,0.018657,0.002941,0.002096,-0.000124,-0.000262,-0.010721,-0.008669,-0.007907,0.004647,-0.003897,-0.009671,0.002659,-0.126606
2015-01-30,-0.009264,0.013808,-0.008637,-0.011006,-0.006821,-0.004075,-0.015769,-0.017151,-0.008543,-0.003805,-0.013396,-0.012014,0.023529,0.028378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.000545,0.002688,0.004362,-0.017795,0.003375,0.001287,0.003364,-0.003230,-0.008377,-0.000007,-0.002462,0.000938,0.010671,0.000749
2024-09-20,0.006323,-0.000596,-0.001923,0.002361,-0.005061,0.002577,-0.000149,-0.000140,-0.005827,-0.011447,-0.001296,0.003873,-0.006050,0.004248
2024-09-27,0.006253,0.003485,-0.006125,0.021917,0.008795,-0.000889,0.004089,0.007682,-0.010445,-0.017034,-0.005565,0.003888,-0.007327,0.006917


In [74]:
# Select SGDNEER_index for the Optimization Period

SGDNEER_index_for_optimization = SGDNEER_index.loc[optimization_start:optimization_end,:]

In [75]:
SGDNEER_index_for_optimization

,MAS_Index,MAS_Index_Change,Matched_Index
Date,,,
2015-01-02,122.80,-0.001139,122.800000
2015-01-09,122.62,-0.001466,122.674326
2015-01-16,122.65,0.000245,122.633329
2015-01-23,122.79,0.001141,122.382861
2015-01-30,122.30,-0.003991,121.849231
...,...,...,...
2024-09-13,139.53,-0.000358,139.687601
2024-09-20,139.62,0.000645,139.646653
2024-09-27,139.74,0.000859,139.763255


In [76]:
# Extract and Initialize Weights

SGD_optimized_weights = SGD_weights.copy()
SGD_optimized_weights = SGD_optimized_weights.loc[optimization_start:optimization_end,:]

In [77]:
SGD_optimized_weights

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-09,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-16,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-23,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-30,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785
2024-09-20,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785
2024-09-27,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785


In [78]:
# Initialize weights as the last row’s weights
initial_weights = SGD_optimized_weights.iloc[-1, :].values.flatten()

In [79]:
initial_weights

array([0.2116259917281864, 0.1040222123069152, 0.018529220821581458,
       0.04720988194909246, 0.061738417893663404, 0.1443845898357622,
       0.1142986402107496, 0.013314513899125078, 0.03034007645883775,
       0.12342104755590033, 0.05827960713187804, 0.023858693508748727,
       0.031192305479997722, 0.017784801219561597], dtype=object)

In [80]:
SGDNEER_index_for_optimization

,MAS_Index,MAS_Index_Change,Matched_Index
Date,,,
2015-01-02,122.80,-0.001139,122.800000
2015-01-09,122.62,-0.001466,122.674326
2015-01-16,122.65,0.000245,122.633329
2015-01-23,122.79,0.001141,122.382861
2015-01-30,122.30,-0.003991,121.849231
...,...,...,...
2024-09-13,139.53,-0.000358,139.687601
2024-09-20,139.62,0.000645,139.646653
2024-09-27,139.74,0.000859,139.763255


In [81]:
# Define the objective function to minimize (total squared error)
def objective_function(initial_weights):
    
    # Calculate the matched index using the same weights for all rows
    SGDNEER_optimized_index = SGDNEER_index_for_optimization.copy()
    SGDNEER_optimized_index["Matched_Index"] = np.nan
    SGDNEER_optimized_index.iloc[0, 2] = SGDNEER_optimized_index.iloc[0, 0]

    # Calculate the matched index using the same weights for all rows
    for row in range(1, len(SGDNEER_optimized_index)):
        # Apply the single weight vector across all rows
        matched_return = np.dot(SGD_optimization_weekly_returns.iloc[row, :], initial_weights)
        
        SGDNEER_optimized_index.iloc[row, -1] = (
            SGDNEER_optimized_index.iloc[row - 1, -1] * (1 + matched_return)
             )

    SGDNEER_optimized_index["Optimized_Index_Change"] = \
        (
            SGDNEER_optimized_index["Matched_Index"]
            /
            SGDNEER_optimized_index["Matched_Index"].shift(1) 
            - 1
        )

    # Calculate the root mean squared error
    squared_error = \
        (
            SGDNEER_optimized_index["MAS_Index_Change"] 
            - 
            SGDNEER_optimized_index["Optimized_Index_Change"]
        ) ** 2
    
    # Calculate the mean of squared errors
    mse = squared_error.mean()
    
    # Calculate the RMSE
    rmse = np.sqrt(mse)

    # Return the sum of squared errors
    return rmse


In [82]:
# Define the equality constraint: Sum of weights = 1 
def weight_sum_constraint(initial_weights):
    # Ensure the sum of the single weight vector is 1
    return np.sum(initial_weights) - 1

In [83]:
# Define the bounds: Each weight must be positive and within ±5% of the initial guess

bounds = [(max(0, w * 0.95), w * 1.05) for w in initial_weights]

In [84]:
# Set up the constraints for the optimizer
constraints = {
    'type': 'eq',  # Equality constraint
    'fun': weight_sum_constraint
}

In [85]:
# https://stackoverflow.com/questions/76122980/fitting-data-with-scipy-optimize-minimize-with-both-constraints-and-bounds

# Use scipy.optimize.minimize to find the optimal weights
result = sco.minimize(
    objective_function, 
    initial_weights, 
    method='SLSQP',  # Use Sequential Least SQuares Programming (SLSQP) since it supports bounds and constraints
    bounds=bounds, 
    constraints=constraints, 
    options={'disp': True, 
             'xtol': 1e-300, 
             'ftol': 1e-300,
             'maxiter': 1000
             }
)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.001154038893311975
            Iterations: 19
            Function evaluations: 297
            Gradient evaluations: 19


In [86]:
# Get the optimized weights from the result
optimized_weights = result.x

In [87]:
optimized_weights 

array([0.20688546, 0.10922332, 0.01945568, 0.04957038, 0.0597471 ,
       0.15160382, 0.10858371, 0.01398024, 0.03185708, 0.11725   ,
       0.05536563, 0.02505163, 0.03275192, 0.01867404])

In [88]:
print("Optimized Weights:\n", optimized_weights)
print("Minimum Squared Error:", result.fun)

Optimized Weights:
 [0.20688546 0.10922332 0.01945568 0.04957038 0.0597471  0.15160382
 0.10858371 0.01398024 0.03185708 0.11725    0.05536563 0.02505163
 0.03275192 0.01867404]
Minimum Squared Error: 0.001154038893311975


# Use the optimized weights to construct the SGDNEER chart

In [89]:
SGD_optimized_weights

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-09,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-16,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-23,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
2015-01-30,0.2154,0.110162,0.016324,0.056322,0.056157,0.139305,0.07667,0.016489,0.031057,0.128403,0.073565,0.026154,0.033241,0.020751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785
2024-09-20,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785
2024-09-27,0.211626,0.104022,0.018529,0.04721,0.061738,0.144385,0.114299,0.013315,0.03034,0.123421,0.05828,0.023859,0.031192,0.017785


In [90]:
# Assign the optimized_weights back to the weekly SGD_optimized_weights dataframe

for row in range(len(SGD_optimized_weights)):
    SGD_optimized_weights.iloc[row,:] = optimized_weights 

In [91]:
SGDNEER_optimized = \
    (
        SGDNEER
        [
            (SGDNEER.index>=optimization_start) 
            & 
            (SGDNEER.index<optimization_end)
        ]
    )


SGDNEER_optimized.rename(columns={"Index":"MAS_Index"},
                     inplace=True)

In [92]:
# Create a column to mimic the MAS SGDNEER index and use the first MAS value as the value of this index

SGDNEER_optimized["Optimized_Index"]=np.nan
SGDNEER_optimized.iloc[0,2]=SGDNEER_optimized.iloc[0,0]

In [93]:
# For every week, multiply the weekly returns with the weights to get the change in SGDNEER

for row in range(1,len(SGDNEER_optimized)):
    SGDNEER_optimized.iloc[row,-1] = \
        (
            SGDNEER_optimized.iloc[row-1,-1] 
            * 
            (
                1+(sum(SGD_optimization_weekly_returns.iloc[row,:] * SGD_optimized_weights.iloc[row,:]))
            )
        )

In [94]:
SGDNEER_optimized["Gap"] = \
    (
        SGDNEER_optimized["Optimized_Index"] / SGDNEER_optimized["MAS_Index"]
    )-1

In [95]:
SGDNEER_optimized.to_csv(path_or_buf="SGDNEER_optimized.csv")

In [96]:
# Plot MAS_Index and Optimized_Index using Plotly

fig = px.line(
    SGDNEER_optimized, 
    x=SGDNEER_optimized.index, 
    y=["MAS_Index", "Optimized_Index"],
    labels={"value": "Index Value", "variable": "Index Type"},
    title="MAS Index vs Optimized Index"
)

# Show the plot
fig.show()

In [97]:
# Plot Gap between Optimized_Index and MAS_Index using Plotly

fig = px.line(
    SGDNEER_optimized, 
    x=SGDNEER_optimized.index, 
    y=["Gap"],
    labels={"Gap": "Index Value", "variable": "Index Type"},
    title="Gap between Optimized Index and MAS Index"
)

# Show the plot
fig.show()

## If satisfied with optimization results, copy the relevant rows in SGD_optimized_weights to the main dataframe SGD_optimized_overall_weights

In [98]:
for row in range(len(SGD_optimized_overall_weights)):
    for row_1 in range(len(SGD_optimized_weights)):
        if SGD_optimized_overall_weights.index[row] == SGD_optimized_weights.index[row_1]:
            SGD_optimized_overall_weights.iloc[row,:] = SGD_optimized_weights.iloc[row_1,:]
            break

In [99]:
SGD_optimized_overall_weights

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-09,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-16,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-23,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-30,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2024-09-20,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2024-09-27,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674


In [100]:
SGD_optimized_overall_weights.to_csv(path_or_buf="SGD_optimized_overall_weights.csv")

# Plot the MAS SGDNEER index, along with center, upper and lower bands

In [101]:
# Record of all MAS actions

# "Date", "Center", "Upper", "Lower", "Slope", "Band", "Comment"

MAS_data = [
    ["2-Jan-02", 99.00, 100.98, 97.02, "0.0%", "2.0%", "Re-centre downwards, at prevailing level of the S$NEER"],
    ["10-Jul-03", 97.20, 99.14, 95.26, "0.0%", "2.0%", "Re-centre downwards, at prevailing level of the S$NEER"],
    ["12-Apr-04", 97.20, 99.14, 95.26, "2.0%", "2.0%", "Increase slope"],
    ["10-Oct-07", 104.28, 106.36, 102.19, "3.0%", "2.0%", "Increase slightly"],
    ["10-Apr-08", 107.80, 109.96, 105.64, "3.0%", "2.0%", "Re-centre upwards to the prevailing level of the S$NEER"],
    ["10-Oct-08", 109.42, 111.61, 107.23, "0.0%", "2.0%", "Reduce to 0%"],
    ["14-Apr-09", 107.43, 109.58, 105.28, "0.0%", "2.0%", "Re-centre downwards to the prevailing level of the S$NEER"],  
    ["14-Apr-10", 109.39, 111.58, 107.20, "2.0%", "2.0%", "Re-center upwards to the prevailing level; Increase slope"],
    ["14-Oct-10", 110.50, 113.82, 107.19, "3.0%", "3.0%", "Increase slightly, Widen Band Slightly"],  
    ["14-Apr-11", 114.00, 117.42, 110.58, "3.0%", "3.0%", "Re-centred upwards, below the prevailing level of the S$NEER"],
    ["14-Oct-11", 115.68, 119.15, 112.21, "1.5%", "3.0%", "Reduced slope"],
    ["13-Apr-12", 116.56, 118.90, 114.23, "2.0%", "2.0%", "Increase slightly, Narrow band"],
    ["28-Jan-15", 123.24, 125.71, 120.78, "1.0%", "2.0%", "Reduced slope"],
    ["14-Oct-15", 124.11, 126.59, 121.63, "0.5%", "2.0%", "Reduce slightly"],
    ["14-Apr-16", 124.41, 126.90, 121.92, "0.0%", "2.0%", "Reduce to 0%"],
    ["13-Apr-18", 124.42, 126.91, 121.93, "0.5%", "2.0%", "Increase slightly"],
    ["12-Oct-18", 124.74, 124.24, 122.25, "1.0%", "2.0%", "Increase slightly"],
    ["14-Oct-19", 125.97, 128.49, 123.45, "0.5%", "2.0%", "Reduce slightly"],
    ["30-Mar-20", 125.00, 127.50, 120.50, "0.0%", "2.0%", "Re-centre downwards, Zero slope"],
    ["14-Oct-21", 125.01, 127.51, 122.51, "0.5%", "2.0%", "Increase slightly"],
    ["25-Jan-22", 125.20, 127.70, 122.70, "1.0%", "2.0%", "Increase slightly"],
    ["14-Apr-22", 128.06, 130.62, 125.50, "1.5%", "2.0%", "Re-center upwards to the prevailing level; Increase slightly"],
    ["14-Jul-22", 130.77, 133.39, 128.15, "1.5%", "2.0%", "Re-center upwards to the prevailing level"],
    ["14-Oct-22", 133.50, 136.17, 130.83, "1.5%", "2.0%", "Re-center upwards to the prevailing level"]
]

In [102]:
# Turn the records into a dataframe

MAS_actions_df = pd.DataFrame(MAS_data, columns=["Date", "Center", "Upper", "Lower", "Slope", "Band", "Comment"])

MAS_actions_df["Date"] = pd.to_datetime(MAS_actions_df["Date"])

MAS_actions_df["Slope"] = MAS_actions_df["Slope"].str.rstrip('%').astype('float') / 100.0

MAS_actions_df["Band"] = MAS_actions_df["Band"].str.rstrip('%').astype('float') / 100.0

In [103]:
MAS_actions_df

,Date,Center,Upper,Lower,Slope,Band,Comment
0,2002-01-02,99.00,100.98,97.02,0.000,0.02,"Re-centre downwards, at prevailing level of th..."
1,2003-07-10,97.20,99.14,95.26,0.000,0.02,"Re-centre downwards, at prevailing level of th..."
2,2004-04-12,97.20,99.14,95.26,0.020,0.02,Increase slope
3,2007-10-10,104.28,106.36,102.19,0.030,0.02,Increase slightly
4,2008-04-10,107.80,109.96,105.64,0.030,0.02,Re-centre upwards to the prevailing level of t...
5,2008-10-10,109.42,111.61,107.23,0.000,0.02,Reduce to 0%
6,2009-04-14,107.43,109.58,105.28,0.000,0.02,Re-centre downwards to the prevailing level of...
7,2010-04-14,109.39,111.58,107.20,0.020,0.02,Re-center upwards to the prevailing level; Inc...
8,2010-10-14,110.50,113.82,107.19,0.030,0.03,"Increase slightly, Widen Band Slightly"
9,2011-04-14,114.00,117.42,110.58,0.030,0.03,"Re-centred upwards, below the prevailing level..."


In [104]:
# Create columns for 

SGDNEER[["Center","Upper","Lower","Slope","Band"]]= np.nan

In [105]:
# Step 1: Assign Center, Slope, and Band for the meeting dates
for row_1 in range(len(MAS_actions_df)):
    # Get the meeting date from MAS_actions_df
    meeting_date = MAS_actions_df.iloc[row_1, MAS_actions_df.columns.get_loc("Date")]

    # Find the index in SGDNEER that is on or immediately after the meeting date
    idx = SGDNEER.index.searchsorted(meeting_date, side='left')

    # Ensure the index is within bounds before assigning
    if idx < len(SGDNEER):
        SGDNEER.iloc[idx, SGDNEER.columns.get_loc("Center")] = MAS_actions_df.iloc[row_1, 1]  # Center
        SGDNEER.iloc[idx, SGDNEER.columns.get_loc("Slope")] = MAS_actions_df.iloc[row_1, 4]  # Slope
        SGDNEER.iloc[idx, SGDNEER.columns.get_loc("Band")] = MAS_actions_df.iloc[row_1, 5]   # Band
        
SGDNEER["Slope"]=SGDNEER["Slope"].ffill()  
SGDNEER["Band"]=SGDNEER["Band"].ffill()  

# Step 2: Calculate Center, Upper, and Lower for all other dates
for row in range(1, len(SGDNEER)):
    # If the Center is NaN, calculate it based on the previous day's Center and slope
    if pd.isna(SGDNEER.iloc[row, SGDNEER.columns.get_loc("Center")]):
        SGDNEER.iloc[row, SGDNEER.columns.get_loc("Center")] = \
            SGDNEER.iloc[row - 1, SGDNEER.columns.get_loc("Center")] * \
            (1 + (SGDNEER.iloc[row, SGDNEER.columns.get_loc("Slope")]* 7 / 365.25))

    # Calculate Upper and Lower bands based on the calculated/assigned Center and Band
    center_value = SGDNEER.iloc[row, SGDNEER.columns.get_loc("Center")]
    band_value = SGDNEER.iloc[row, SGDNEER.columns.get_loc("Band")]

    SGDNEER.iloc[row, SGDNEER.columns.get_loc("Upper")] = center_value * (1 + band_value)
    SGDNEER.iloc[row, SGDNEER.columns.get_loc("Lower")] = center_value * (1 - band_value)

In [106]:
SGDNEER.loc["2011-04-01":,:]

,Index,MAS_Index_Change,Center,Upper,Lower,Slope,Band
Date,,,,,,,
2011-04-01,115.36,0.003218,112.034888,115.395935,108.673841,0.030,0.03
2011-04-08,115.32,-0.000347,112.099302,115.462281,108.736323,0.030,0.03
2011-04-15,115.61,0.002515,114.000000,117.420000,110.580000,0.030,0.03
2011-04-22,116.30,0.005968,114.065544,117.487510,110.643578,0.030,0.03
2011-04-29,116.45,0.001290,114.131126,117.555060,110.707192,0.030,0.03
...,...,...,...,...,...,...,...
2024-09-13,139.53,-0.000358,137.392910,140.140769,134.645052,0.015,0.02
2024-09-20,139.62,0.000645,137.432407,140.181055,134.683759,0.015,0.02
2024-09-27,139.74,0.000859,137.471916,140.221354,134.722477,0.015,0.02


In [107]:
SGDNEER.to_csv(path_or_buf="SGDNEER.csv")

In [108]:
# Create the figure
fig = go.Figure()

# Add the SGDNEER Index line
fig.add_trace(go.Scatter(
    x=SGDNEER.index,
    y=SGDNEER["Index"],
    mode='lines',
    name='Index',
    line=dict(color='blue')
))

# Add the Center line
fig.add_trace(go.Scatter(
    x=SGDNEER.index,
    y=SGDNEER["Center"],
    mode='lines',
    name='Center',
    line=dict(color='green')
))

# Add the Upper band line
fig.add_trace(go.Scatter(
    x=SGDNEER.index,
    y=SGDNEER["Upper"],
    mode='lines',
    name='Upper Band',
    line=dict(color='red', dash='dash')
))

# Add the Lower band line
fig.add_trace(go.Scatter(
    x=SGDNEER.index,
    y=SGDNEER["Lower"],
    mode='lines',
    name='Lower Band',
    line=dict(color='red', dash='dash')
))

# Update layout
fig.update_layout(
    title='SGDNEER Index with Center, Upper, and Lower Bands',
    xaxis_title='Date',
    yaxis_title='Value',
    legend_title='Legend',
    hovermode='x unified'  # Show hover information for all traces on x-axis
)

# Show the plot
fig.show()

# Plot the Optimized Daily SGDNEER index, along with center, upper and lower bands

In [109]:
SGD_daily_returns = \
    (
        np.log(SGD_base_prices/SGD_base_prices.shift(1))
    )

In [110]:
SGD_daily_returns = SGD_daily_returns[SGD_daily_returns.index>=start_analysis]

In [111]:
SGD_daily_returns

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-01,-0.001936,0.003293,-0.002699,-0.000137,-0.004312,-0.001274,0.019381,-0.001443,-0.001753,-0.001364,-0.004354,-0.007473,-0.000382,0.003683
2015-01-02,0.000091,0.000853,0.000169,0.001736,0.000914,0.000091,0.000091,-0.000290,0.000669,0.007780,0.000091,0.000091,-0.000727,0.000081
2015-01-05,-0.007878,0.004019,0.011226,-0.003184,0.007907,-0.007878,-0.007878,-0.005507,-0.000787,-0.000388,-0.000640,-0.003919,0.005687,0.003823
2015-01-06,-0.000038,0.000584,0.001703,-0.008451,-0.002290,0.001946,0.011282,0.004532,-0.005579,0.005298,0.011514,0.000910,-0.003999,0.000360
2015-01-07,0.000210,0.005556,0.008128,-0.006107,-0.009945,-0.000821,0.000366,-0.000725,-0.002435,0.001190,0.001002,0.003835,0.002637,0.005266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-17,-0.002852,-0.000862,0.002959,0.000342,-0.002104,-0.002767,-0.003971,-0.001276,-0.006252,-0.006456,-0.002512,-0.003053,-0.000276,0.000343
2024-10-18,-0.000624,0.002413,-0.002588,0.002873,0.003655,0.000022,-0.002711,-0.002218,-0.001619,0.003212,-0.006474,-0.000452,-0.005836,0.000104
2024-10-21,0.003193,-0.000408,0.000365,-0.000143,0.002434,0.000072,0.002382,0.000068,0.001563,0.000172,0.001514,0.003371,0.000644,0.002038


In [112]:
SGD_optimized_overall_weights

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-02,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-09,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-16,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-23,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-30,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-13,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2024-09-20,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2024-09-27,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674


In [113]:
# Create the DataFrame with the same index and columns as SGD_daily_returns
SGD_daily_optimized_weights = pd.DataFrame(index=SGD_daily_returns.index,
                                           columns=SGD_daily_returns.columns)

# Loop through the daily index and assign the corresponding overall weight
for row in range(len(SGD_daily_optimized_weights)):
    # Get the current daily index value
    current_date = SGD_daily_optimized_weights.index[row]

    # Find the matching range in the overall weights
    for row_1 in range(len(SGD_optimized_overall_weights) - 1):
        if (current_date >= SGD_optimized_overall_weights.index[row_1]) and \
           (current_date < SGD_optimized_overall_weights.index[row_1 + 1]):
            # Assign the corresponding weights for the matching range
            SGD_daily_optimized_weights.iloc[row, :] = SGD_optimized_overall_weights.iloc[row_1, :]
            break
    else:
        # If the date is beyond the last range, assign the last row's weights
        SGD_daily_optimized_weights.iloc[row, :] = SGD_optimized_overall_weights.iloc[-1, :]

In [114]:
SGD_daily_optimized_weights

,SGDUSD,SGDEUR,SGDGBP,SGDJPY,SGDKRW,SGDCNY,SGDTWD,SGDPHP,SGDTHB,SGDMYR,SGDIDR,SGDINR,SGDAUD,SGDCHF
Date,,,,,,,,,,,,,,
2015-01-01,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-02,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-05,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-06,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2015-01-07,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-17,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2024-10-18,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674
2024-10-21,0.206885,0.109223,0.019456,0.04957,0.059747,0.151604,0.108584,0.01398,0.031857,0.11725,0.055366,0.025052,0.032752,0.018674


In [115]:
SGD_daily_index = pd.DataFrame(index=SGD_daily_optimized_weights.index,
                               columns = ["SGDNEER"])



In [116]:
# Assign first value of MAS SGDNEER index to our index
SGD_daily_index.iloc[0,0] = SGDNEER_index.iloc[0,0]

In [117]:
# For every day, multiply the daily returns with the daily weights to get the change in SGDNEER

for row in range(1,len(SGD_daily_index)):
    SGD_daily_index.iloc[row,-1] = \
        (
            SGD_daily_index.iloc[row-1,-1] 
            * 
            (
                1+(sum(SGD_daily_returns.iloc[row,:] * SGD_daily_optimized_weights.iloc[row,:]))
            )
        )

In [118]:
# Create columns for 

SGD_daily_index[["Center","Upper","Lower","Slope","Band"]]= np.nan

In [119]:
# Step 1: Assign Center, Slope, and Band for the meeting dates
for row_1 in range(len(MAS_actions_df)):
    # Get the meeting date from MAS_actions_df
    meeting_date = MAS_actions_df.iloc[row_1, MAS_actions_df.columns.get_loc("Date")]

    # Find the index in SGD_daily_index that is on or immediately after the meeting date
    idx = SGD_daily_index.index.searchsorted(meeting_date, side='left')

    # Ensure the index is within bounds before assigning
    if idx < len(SGD_daily_index):
        SGD_daily_index.iloc[idx, SGD_daily_index.columns.get_loc("Center")] = MAS_actions_df.iloc[row_1, 1]  # Center
        SGD_daily_index.iloc[idx, SGD_daily_index.columns.get_loc("Slope")] = MAS_actions_df.iloc[row_1, 4]  # Slope
        SGD_daily_index.iloc[idx, SGD_daily_index.columns.get_loc("Band")] = MAS_actions_df.iloc[row_1, 5]   # Band
        
SGD_daily_index["Slope"]=SGD_daily_index["Slope"].ffill()  
SGD_daily_index["Band"]=SGD_daily_index["Band"].ffill()  

# Step 2: Set Center, Upper, and Lower for the first date in the index

SGD_daily_index.iloc[0, SGD_daily_index.columns.get_loc("Center")] = 123.0787  # Center
SGD_daily_index.iloc[0, SGD_daily_index.columns.get_loc("Upper")] = 125.5403   # Center
SGD_daily_index.iloc[0, SGD_daily_index.columns.get_loc("Lower")] = 120.6172   # Center

# Step 3: Calculate Center, Upper, and Lower for all other dates
for row in range(1, len(SGD_daily_index)):
    # If the Center is NaN, calculate it based on the previous day's Center and slope
    if pd.isna(SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Center")]):
        SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Center")] = \
            SGD_daily_index.iloc[row - 1, SGD_daily_index.columns.get_loc("Center")] * \
            (1 + (SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Slope")] / 252))  #Number of trading days in a year

    # Calculate Upper and Lower bands based on the calculated/assigned Center and Band
    center_value = SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Center")]
    band_value = SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Band")]

    SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Upper")] = center_value * (1 + band_value)
    SGD_daily_index.iloc[row, SGD_daily_index.columns.get_loc("Lower")] = center_value * (1 - band_value)

In [120]:
SGD_daily_index

,SGDNEER,Center,Upper,Lower,Slope,Band
Date,,,,,,
2015-01-01,122.8,123.078700,125.540300,120.617200,0.020,0.02
2015-01-02,122.946602,123.088468,125.550238,120.626699,0.020,0.02
2015-01-05,122.610808,123.098237,125.560202,120.636272,0.020,0.02
2015-01-06,122.867823,123.108007,125.570167,120.645847,0.020,0.02
2015-01-07,122.894205,123.117777,125.580133,120.655422,0.020,0.02
...,...,...,...,...,...,...
2024-10-17,138.12074,137.721221,140.475645,134.966797,0.015,0.02
2024-10-18,138.105383,137.729419,140.484007,134.974830,0.015,0.02
2024-10-21,138.288973,137.737617,140.492369,134.982864,0.015,0.02


In [121]:
#SGD_daily_index.to_csv(path_or_buf="SGD_daily_index.csv")

In [122]:
# Create the figure
fig = go.Figure()

# Add the SGDNEER line
fig.add_trace(go.Scatter(
    x=SGD_daily_index.index,
    y=SGD_daily_index["SGDNEER"],
    mode='lines',
    name='Index',
    line=dict(color='blue')
))

# Add the Center line
fig.add_trace(go.Scatter(
    x=SGD_daily_index.index,
    y=SGD_daily_index["Center"],
    mode='lines',
    name='Center',
    line=dict(color='green')
))

# Add the Upper band line
fig.add_trace(go.Scatter(
    x=SGD_daily_index.index,
    y=SGD_daily_index["Upper"],
    mode='lines',
    name='Upper Band',
    line=dict(color='red', dash='dash')
))

# Add the Lower band line
fig.add_trace(go.Scatter(
    x=SGD_daily_index.index,
    y=SGD_daily_index["Lower"],
    mode='lines',
    name='Lower Band',
    line=dict(color='red', dash='dash')
))

# Update layout
fig.update_layout(
    title='SGDNEER Index with Center, Upper, and Lower Bands',
    xaxis_title='Date',
    yaxis_title='Value',
    legend_title='Legend',
    hovermode='x unified'  # Show hover information for all traces on x-axis
)

# Show the plot
fig.show()